# Training an intents classification model

Define constants

In [1]:
from os import getcwd, path
import sys
# import matplotlib.pyplot as plt
import numpy as np

BASE_PATH = path.dirname(getcwd())
sys.path.append(BASE_PATH)

DATA_UTILS = path.join(BASE_PATH, 'common/data_utils.py')
TRAIN_PATH = path.join(BASE_PATH, 'kc_data.json')
CLASSES_FILE = path.join(BASE_PATH, 'classes.json')

In [2]:
exec(open(DATA_UTILS).read())

Use functions from the utils to extract and preprocess the training data
Refer to `kc_data.json` for the sample data format
`get_data_pairs` is then used to parse data into a tuple of `([list_of_sentences], [list_of_labels])`

In [3]:
X_data, y_data = get_data_pairs(data_from_json(TRAIN_PATH))

classes = list(set(y_data))
num_classes = len(classes)

X_train = X_data
y_train = np.array([
    classes.index(item)
    for item in y_data
], dtype='int32')

In [4]:
print('Number of samples: %s, classes: %s' % (len(X_train), num_classes))

Number of samples: 10256, classes: 1427


Save the classes descriptions as json

In [5]:
import torch
torch.__version__

'1.0.0'

Start training the classification model and save

In [6]:
from text_classification.sif_ovr.model import OvrClassifierWrapper
from text_classification.sif_ovr.train import OvrClassifierLearner
from common.callbacks import PrintLoggerCallback

model = OvrClassifierWrapper({'num_classes': num_classes})
learner = OvrClassifierLearner(model)

In [7]:
learner.fit(
    training_data=(X_train, y_train),
    batch_size=64,
    epochs=100,
    callbacks=[PrintLoggerCallback(log_every=5)]
)

17m 35s (- 334m 5s) (5 5%) - loss: 63094.6957 - accuracy: 0.0013
33m 21s (- 300m 10s) (10 10%) - loss: 63094.6897 - accuracy: 0.0013
48m 7s (- 272m 40s) (15 15%) - loss: 63094.6962 - accuracy: 0.0013
63m 41s (- 254m 47s) (20 20%) - loss: 63094.6913 - accuracy: 0.0013
80m 44s (- 242m 12s) (25 25%) - loss: 63094.6936 - accuracy: 0.0013
94m 49s (- 221m 14s) (30 30%) - loss: 63094.6961 - accuracy: 0.0013
108m 35s (- 201m 40s) (35 35%) - loss: 63094.6869 - accuracy: 0.0013


Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
/Users/2359media/Documents/botbot-nlp/common/wrappers.py:643: UserWarning: Training aborted
  warnings.warn('Training aborted')
  File "/Users/2359media/anaconda/envs/botbot-nlp/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/Users/2359media/anaconda/envs/botbot-nlp/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/2359media/anaconda/envs/botbot-nlp/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/2359media/anaconda/envs/botbot-nlp/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/Users/2359media/anaconda/envs/botbot-nlp/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
 

In [8]:
torch.topk(model(['good morning!']), 50)

AttributeError: 'OvrClassifierWrapper' object has no attribute 'topk'

In [ ]:
model(['the earth is flat'])

In [ ]:
classes[230]